# Algoritmos Genéticos

## Exercício 1

In [1]:
# Instalação de dependências
!pip install deap

In [0]:
# Biblioteca de algoritmos genéticos
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

In [0]:
import array
import numpy
import random

from typing import Callable
from typing import List
from typing import Optional
from typing import Tuple

from attr import attrs

from pprint import pprint


In [0]:
class AG:
    """Implementação do algoritmo genético"""

    @attrs(auto_attribs=True, slots=True)
    class Config:
        """Parâmetros de configuração do algoritmo genético."""
        
        # Parâmetros gerais 
        max_geracoes: int = 10  # Número máximo de gerações

        tam_individuo: int = 10  # Tamanho do cromossomo
        tam_populacao: int = 100  # Tamanho da população

        # Parâmetros para seleção dos pais
        tam_torneio: int = 3

        # Parâmetros de crossover e mutação
        taxa_crossover: float = 1.0
        taxa_mutacao: float = 0.1
            
        # Parâmetros para seleção da geração seguinte
        taxa_elitismo: float = 0.3

        # Configuração do cromossomo
        fenotipo: Tuple[int, int] = (0, 1)

        # Função de avaliação
        fitness: Callable[[List[int]], Tuple[float]] = lambda v: (sum(v),)
        
        # Função de ajuste
        feasible: Optional[Callable[[list], bool]] = None
        distance: Optional[Callable[[list], float]] = None

    def executar(self, config: Config):
        """Executa o algoritmo genético .

        :param config: parâmetros de configuração do algoritmo
        :type config: AG.Config
        :return: melhor solução encontrada
        :rtype: List[int]
        """
        toolbox = base.Toolbox()

        # Estrutura dos cromossomos
        creator.create("Fitness", base.Fitness, weights=(1.0,))
        creator.create("Individual", array.array, typecode="b", fitness=creator.Fitness)

        # Estrutura dos cromossomos e da população
        toolbox.register("attr_bool", random.randint, config.fenotipo[0], config.fenotipo[1])
        toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=config.tam_individuo)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # Escolha do algoritmo de mutação
        toolbox.register("mutate", tools.mutFlipBit, indpb=config.taxa_mutacao)

        # Escolha do algoritmo de crossover
        # toolbox.register("mate", tools.cxOnePoint, indpb=config.taxa_crossover)
        # toolbox.register("mate", tools.cxTwoPoint, indpb=config.taxa_crossover)
        toolbox.register("mate", tools.cxUniform, indpb=config.taxa_crossover)

        # Escolha do algoritmo de seleção
        # toolbox.register("select", tools.selRoulette)
        toolbox.register("select", tools.selTournament, tournsize=config.tam_torneio)

        # Registra a função de avaliação
        toolbox.register("evaluate", config.fitness)
        
        # Aplica as restrições de penalidade
        if config.feasible and config.distance:
            toolbox.decorate("evaluate", tools.DeltaPenalty(config.feasible, 1000, config.distance))

        # 1. Geração da população inicial
        populacao = toolbox.population(n=config.tam_populacao)
                
        # Estatísticas
        stats_fit = tools.Statistics(key=lambda ind: ind.fitness.values)
        stats = tools.MultiStatistics(fitness=stats_fit)
        stats.register("estatisticas", lambda v: self.__estatisticas(config, tools.selBest(populacao, k=1)[0]))
        # stats.register("cong", lambda v: congelamento)

        logbook = tools.Logbook()
        logbook.header = ['gen'] + (stats.fields if stats else [])

        record = stats.compile(populacao) if stats is not None else {}
        logbook.record(gen=0, **record)
        print(logbook.stream)

        # 2. Inicia o processo de evolução
        for gen in range(1, config.max_geracoes + 1):
        
            # 3. Seleção dos pais para cruzamento
            pais = toolbox.select(populacao, len(populacao))
            
            # 4. Crossover e mutação
            filhos = self.__cruzamento(toolbox, pais)
            
            # 5. Seleciona os individuos para geração seguinte
            elitismo = int(round(config.taxa_elitismo * len(populacao)))
            populacao = sorted(populacao + filhos, key=lambda x: x.fitness, reverse=True)

            melhores = populacao[:elitismo]
            selecionados = toolbox.select(populacao[elitismo:], config.tam_populacao - elitismo)
            
            # 6. Nova população
            populacao[:] = melhores + selecionados
            
            # TODO: Verificar condição de congelamento
            melhor = tools.selBest(populacao, k=1)

            # Update the statistics with the new population
            record = stats.compile(populacao) if stats is not None else {}
            logbook.record(gen=gen, **record)
            print(logbook.stream)

        # Retorna a melhor solução
        return tools.selBest(populacao, k=1)[0]

    
    def __cruzamento(self, toolbox, pais):
        filhos = []

        for i in range(0, len(pais), 2):
            # Seleciona dois indivíduos e aplica o crossover
            ind1, ind2 = list(map(toolbox.clone, (pais[i], pais[i + 1])))
            ind1, ind2 = toolbox.mate(ind1, ind2)

            # Aplica mutacao nos dois individuos
            toolbox.mutate(ind1)
            toolbox.mutate(ind2)

            # Recalcula o fitness dos filhos
            toolbox.map(toolbox.evaluate, ind1)
            toolbox.map(toolbox.evaluate, ind2)

            ind1.fitness.values = toolbox.evaluate(ind1)
            ind2.fitness.values = toolbox.evaluate(ind2)

            filhos.append(ind1)
            filhos.append(ind2)

        return filhos
    
    def __estatisticas(self, config, individuo):
        individuo = [int(x) for x in individuo]
        return f'{config.fitness(individuo)[0]:10.2f} : {individuo}'


In [24]:
# Função de fitness - maximiza o numero de 1s
def fit_maximiza_um(individuo):
    return sum(individuo),

# Parâmetros de configuração do AG
config = AG.Config(
    fitness=fit_maximiza_um
)

ag = AG()
solucao = ag.executar(config)

print()
print(f'melhor solução: {[x for x in solucao]} - {config.fitness(solucao)[0]}')

   	                      fitness                      
   	---------------------------------------------------
gen	estatisticas                               	gen
0  	      4.00 : [0, 1, 1, 1, 0, 0, 0, 1, 0, 0]	0  
1  	      9.00 : [1, 1, 1, 1, 0, 1, 1, 1, 1, 1]	1  
2  	      9.00 : [1, 1, 1, 1, 0, 1, 1, 1, 1, 1]	2  
3  	     10.00 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]	3  
4  	     10.00 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]	4  
5  	     10.00 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]	5  
6  	     10.00 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]	6  
7  	     10.00 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]	7  
8  	     10.00 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]	8  
9  	     10.00 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]	9  
10 	     10.00 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]	10 

melhor solução: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] - 10


/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [43]:
# Função de fitness - problema dos cartões
def fit_cartoes(individuo):

    # função enumerate(): cria uma enumeração dos itens de uma lista
    # exemplo:
    #     lista = ["eat","sleep","repeat"] 
    #     print([x for x in enumerate(lista)])
    #     > [(0, 'eat'), (1, 'sleep'), (2, 'repeat')]
    #     print([x for x in enumerate(lista, 2)])
    #     > [(2, 'eat'), (3, 'sleep'), (4, 'repeat')]

    # Soma todas as posicoes que contem 0
    soma = sum([i for i, j in enumerate(individuo, 1) if j == 0])

    # Multiplica todas as posicoes que contem 1
    produto = numpy.prod([i for i, j in enumerate(individuo, 1) if j == 1])
    
    # Retorna o valor negativo (minimização)
    return -(abs(36 - soma) + abs(360 - produto)),

# Parâmetros de configuração do AG
config = AG.Config(
    fitness=fit_cartoes
)

ag = AG()
solucao = ag.executar(config)

print()
print(f'cartões da pilha soma....: {[i for i, j in enumerate(solucao, 1) if j == 0]} = {sum([i for i, j in enumerate(solucao, 1) if j == 0])}')
print(f'cartões da pilha produto.: {[i for i, j in enumerate(solucao, 1) if j == 1]} = {numpy.prod([i for i, j in enumerate(solucao, 1) if j == 1])}')
print()


   	                      fitness                      
   	---------------------------------------------------
gen	estatisticas                               	gen
0  	  -3011.00 : [1, 1, 0, 1, 0, 1, 1, 0, 0, 1]	0  
1  	    -13.00 : [0, 0, 0, 0, 1, 0, 1, 0, 0, 1]	1  
2  	    -13.00 : [0, 0, 0, 0, 1, 0, 1, 0, 0, 1]	2  
3  	     -5.00 : [1, 0, 0, 1, 0, 0, 0, 0, 1, 1]	3  
4  	     -5.00 : [1, 0, 0, 1, 0, 0, 0, 0, 1, 1]	4  
5  	     -5.00 : [1, 0, 0, 1, 0, 0, 0, 0, 1, 1]	5  
6  	     -2.00 : [1, 1, 0, 1, 1, 0, 0, 0, 1, 0]	6  
7  	     -2.00 : [1, 1, 0, 1, 1, 0, 0, 0, 1, 0]	7  
8  	     -2.00 : [1, 1, 0, 1, 1, 0, 0, 0, 1, 0]	8  
9  	     -2.00 : [1, 1, 0, 1, 1, 0, 0, 0, 1, 0]	9  
10 	     -2.00 : [1, 1, 0, 1, 1, 0, 0, 0, 1, 0]	10 

cartões da pilha soma....: [3, 6, 7, 8, 10] = 34
cartões da pilha produto.: [1, 2, 4, 5, 9] = 360



/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
